In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets  # Import datasets
from PIL import Image
import os
import glob

In [ ]:
# --- Custom Dataset for FashionMNIST (from a directory) ---
# This is kept for flexibility, in case you DO want to use a custom directory structure later.

class FashionMNISTDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))  # Assuming subfolders are class names
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.imgs = []
        self.labels = []

        for class_name in self.classes:
            class_dir = os.path.join(self.root_dir, class_name)
            for img_path in glob.glob(os.path.join(class_dir, "*.png")):  # Adjust extension if needed
                self.imgs.append(img_path)
                self.labels.append(self.class_to_idx[class_name])

        # Ensure at least some samples are loaded.
        if len(self.imgs) == 0:
            raise RuntimeError(f"Found 0 images in subfolders of: {root_dir}\n"
                               f"Supported image extensions are: .png, .jpg, .jpeg")

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        image = Image.open(img_path).convert("L")  # Convert to grayscale (1 channel)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# --- Network in Network (NIN) Model ---

class MlpConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, num_layers=2):
        super(MlpConvLayer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        layers = []
        current_channels = in_channels
        for _ in range(num_layers - 1):  # All except last layer
            layers.append(nn.Conv2d(current_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding))
            layers.append(nn.BatchNorm2d(out_channels))  # Add BatchNorm
            layers.append(nn.ReLU())
            current_channels = out_channels

        #Last Layer
        layers.append(nn.Conv2d(current_channels, out_channels, kernel_size=1, stride=1, padding=0))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU())

        self.mlpconv = nn.Sequential(*layers)


    def forward(self, x):
        return self.mlpconv(x)

class NIN(nn.Module):
    def __init__(self, num_classes=10):
        super(NIN, self).__init__()

        self.features = nn.Sequential(
            MlpConvLayer(1, 192, kernel_size=5, stride=1, padding=2, num_layers=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Dropout(0.5),
            MlpConvLayer(192, 160, kernel_size=5, stride=1, padding=2, num_layers=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Dropout(0.5),
            MlpConvLayer(160, 96, kernel_size=5, stride=1, padding=2, num_layers=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Dropout(0.5),
            MlpConvLayer(96, num_classes, kernel_size=1, stride=1, padding=0, num_layers=1)
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # Global Average Pooling
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x)
        x = self.flatten(x)
        return x


In [ ]:
# --- Training Function ---

def train(model, train_loader, criterion, optimizer, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (i + 1) % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}")
                running_loss = 0.0

In [ ]:


# --- Testing Function ---

def test(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy on test set: {100 * correct / total:.2f}%")
    return 100 * correct / total


In [ ]:
# --- Main Execution ---

if __name__ == "__main__":
    # --- Hyperparameters and Device Setup ---
    batch_size = 128
    learning_rate = 0.01
    epochs = 30
    num_classes = 10
    data_root = r"C:\Users\Debojyoti Das\data\FashionMNIST"   # Still keep this for potential custom use.

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # --- Data Transformations ---
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # --- Data Loading (Simplified) ---
    # Use torchvision.datasets.FashionMNIST directly.  Much simpler and handles downloading.
    train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)


    # --- Data Loaders ---
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


    # --- Model, Criterion, Optimizer ---
    model = NIN(num_classes=num_classes).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

    # --- Training Loop ---
    train(model, train_loader, criterion, optimizer, device, epochs)

    # --- Test ---
    test(model, test_loader, device)

    #save
    torch.save(model.state_dict(), 'nin_fmnist.pth')

    #Example of loading:
    loaded_model = NIN(num_classes=num_classes)
    loaded_model.load_state_dict(torch.load('nin_fmnist.pth'))
    loaded_model.to(device) #put in device
    loaded_model.eval() #important to put into eval mode after loading
    test(loaded_model, test_loader, device)